In [1]:
from __future__ import print_function
import os
import numpy as np
from numpy import newaxis
from sklearn import preprocessing
np.random.seed(1337)

import keras
from keras.utils.np_utils import to_categorical
from keras.layers import Dense, Input, Flatten
from keras.layers import Dropout
from keras.models import Model,Sequential
import sys

import pandas as pd
import random

Using TensorFlow backend.


# Reading in data

In [2]:
print("Data read started...")
nback = pd.read_csv("result_nback_3.csv")
nback = nback.as_matrix()
print ("Data read finished.")

print("Data read started...")
tetris = pd.read_csv("result_tetris.csv")
tetris = tetris.as_matrix()
print ("Data read finished.")

print(nback.shape, tetris.shape)

Data read started...
Data read finished.
Data read started...
Data read finished.
(522, 11) (603, 11)


# Eliminate EEG data

In [3]:
for i in range (1,5):
    nback = np.delete(nback, 1, 1) 
    tetris = np.delete(tetris, 1, 1)

print(nback.shape, tetris.shape)

(522, 7) (603, 7)


In [4]:
nback[1]

array(['3back', 1318300.0, 1839.9, 673.37, 4.746279069767441, 371.67,
       522.54], dtype=object)

# Dictionary for the levels

In [5]:
level_nback=["1back","2back","3back"]
level_tetris=["tetris1","tetris2","tetris3"]

level_n2int = dict((p, i) for i, p in enumerate(level_nback))
int2level_n = dict((i, p) for i, p in enumerate(level_nback))

level_t2int = dict((p, i) for i, p in enumerate(level_tetris))
int2level_t = dict((i, p) for i, p in enumerate(level_tetris))

# Z normalize

In [6]:
for i in range (1,7):
    nback[:, i] = preprocessing.scale(nback[:, i])
    tetris[:, i] = preprocessing.scale(tetris[:, i])

/home/amoro/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
/home/amoro/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
/home/amoro/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
/home/amoro/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
/home/amoro/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConve

In [7]:
for i in range(len(nback)):
    nback[i, 0] = level_n2int[nback[i, 0]]
    
for i in range(len(tetris)):
    tetris[i, 0] = level_t2int[tetris[i, 0]]

x_nback = nback[:, 1:]
y_nback = nback[:, 0]
x_tetris = tetris[:, 1:]
y_tetris = tetris[:, 0]

# One-hot encoding

In [8]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()
nback_one_hot = ohe.fit_transform(y_nback.reshape(-1,1)).toarray()
tetris_one_hot = ohe.fit_transform(y_tetris.reshape(-1,1)).toarray()
print(nback_one_hot, tetris_one_hot)

[[ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 ..., 
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]] [[ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 ..., 
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]]


# Shuffle data

In [9]:
indices = np.arange(nback.shape[0])
np.random.shuffle(indices)

x_nback = x_nback[indices]
nback_one_hot = nback_one_hot[indices]

indices = np.arange(tetris.shape[0])
np.random.shuffle(indices)

x_tetris = x_tetris[indices]
tetris_one_hot = tetris_one_hot[indices]

# Divide into train, validation and test data

In [10]:
nb_train = int(len(x_nback)*0.85)

x_train = x_nback[0:nb_train]
y_train = nback_one_hot[0:nb_train]

x_valid = x_nback[nb_train:]
y_valid = nback_one_hot[nb_train:]

x_test = x_tetris[0:100]
y_test = tetris_one_hot[0:100]

In [11]:
print(x_train.shape, y_train.shape, x_valid.shape, y_valid.shape, x_test.shape, y_test.shape)

(443, 6) (443, 3) (79, 6) (79, 3) (100, 6) (100, 3)


# Build the net

In [12]:
model = Sequential()

model.add(Dense(200, input_shape=(6,)))
model.add(Dropout(0.25))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(3, activation='softmax'))

In [13]:
batch_size = 50
epochs = 150
earlyStopping=keras.callbacks.EarlyStopping(monitor='val_loss', patience=15, verbose=0, mode='auto')

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(x_train, y_train, nb_epoch=epochs,batch_size=batch_size, 
          callbacks=[earlyStopping], shuffle=True, validation_data = (x_valid, y_valid))

Train on 443 samples, validate on 79 samples
Epoch 1/150
443/443 [==============================] - 0s - loss: 1.0041 - acc: 0.5102 - val_loss: 0.8811 - val_acc: 0.6456
Epoch 2/150
443/443 [==============================] - 0s - loss: 0.9024 - acc: 0.6208 - val_loss: 0.8326 - val_acc: 0.6329
Epoch 3/150
443/443 [==============================] - 0s - loss: 0.8694 - acc: 0.6208 - val_loss: 0.8360 - val_acc: 0.6456
Epoch 4/150
443/443 [==============================] - 0s - loss: 0.8236 - acc: 0.6591 - val_loss: 0.8349 - val_acc: 0.6582
Epoch 5/150
443/443 [==============================] - 0s - loss: 0.8205 - acc: 0.6456 - val_loss: 0.8110 - val_acc: 0.6582
Epoch 6/150
443/443 [==============================] - 0s - loss: 0.8044 - acc: 0.6546 - val_loss: 0.7990 - val_acc: 0.6582
Epoch 7/150
443/443 [==============================] - 0s - loss: 0.7927 - acc: 0.6614 - val_loss: 0.7859 - val_acc: 0.6582
Epoch 8/150
443/443 [==============================] - 0s - loss: 0.7886 - acc: 0.6614 

In [14]:
model.evaluate(x_test, y_test, batch_size=batch_size)

100/100 [==============================] - 0s     


[6.3876605033874512, 0.21000000089406967]

In [15]:
from sklearn.metrics import classification_report,confusion_matrix

y_pred = model.predict_classes(x_test)
print(y_pred)

p=model.predict_proba(x_test)

target_names = ['class 0(level1)', 'class 1(level2)', 'class 2(level3)']
print(classification_report(np.argmax(y_test,axis=1), y_pred,target_names=target_names))
print(confusion_matrix(np.argmax(y_test,axis=1), y_pred))

 32/100 [========>.....................] - ETA: 0s[2 2 2 0 2 0 2 0 0 2 2 1 2 1 2 1 2 1 2 2 0 2 2 2 2 0 1 2 0 2 0 0 0 2 2 1 1
 2 0 1 1 2 1 2 2 0 1 0 2 0 1 2 1 2 2 1 0 0 2 2 2 0 1 2 1 1 0 0 0 1 0 2 2 2
 0 1 2 0 2 2 1 0 2 1 2 2 2 0 2 2 2 0 2 0 0 2 2 1 2 0]
 32/100 [========>.....................] - ETA: 0s                 precision    recall  f1-score   support

class 0(level1)       0.07      0.06      0.06        35
class 1(level2)       0.50      0.29      0.37        38
class 2(level3)       0.16      0.30      0.21        27

    avg / total       0.26      0.21      0.22       100

[[ 2  6 27]
 [13 11 14]
 [14  5  8]]
